### Endomorphism API

This notebook exists to explain how to us the current functionality (At present living in the
`features/nodes-in-trees` branch) to manipulate the graphs we are working with.

In [1]:
from function import Node, Tree, tipless, tree_tidy, tree_to_nodes, nodes_to_tree, verify_mapping

The base representation of a tree is a dict whose keys are node identities (currently integers, but there
is no reason why nodes shold not be arbitrarily named) and whose values are lists of ids of "child" nodes.
(We can straighten the terminology out later). For convenience of programming these "d-tree" representations
can be transformed into a node-based "n-tree" representation.

You can verify that a dict represents a mapping by calling `verify_mapping` with a dict argument.
If the result is a null string the mapping checks out, allowing us to detect illegal mappings before
starting to process them.

It strikes me that `mapping_errors` might be a better name for readbility,
since we would then naturally be able to write code 

    errs = mapping_errors(my_mapping)
    if errs:
        <take appropriate action>


In [2]:
verify_mapping({1:[2]}, 1)

''

In [3]:
mapping_errors = verify_mapping
errs = mapping_errors({1:[2]}, 2)
if errs:
    print(f"Mapping discrepancy: {errs}")

Mapping discrepancy: Keys [1] Values [2]


This mechanism to ensure we are dealing with vaid data should be used on all inputs.
There's nothing worse than spending hourse trying to find out what's wrong with your code,
only to discover the error is in the data. [_The function needs rather more rigorous testing_].

This function is used in the function that converts a d-tree (one defined in a dictionary) to an n-tree
(one defined by relationships between nodes).

In [4]:
ntree = tree_to_nodes({1:[2]}, 2)

ValueError: Error in mapping: Keys [1] Values [2]

The error message might be clearer, but at least we aren't working with duff data!
It's a debatable question whether to build the checking in or leave it to the user
to perform. What happens with good data?

In [5]:
ntree_1 = tree_to_nodes({1:[2]}, 1)

We now have a node-based tree.
What does it look like?

In [6]:
ntree_1

<Node 1 - 0 inessential(s), 1 child(ren)>

One immediate advantage of this representation is
that the root does not need to be passed separately
from the conectivity data.
The output abive is actually just the representation of the root node of the tree.
Trees have a method to print them out in full, though, in a way that makes the
structure relatively readable.

In [7]:
print(ntree_1.pretty())

Node 1: ()
| Node 2: ()


You can see that the children of a node appear indented one level.
Their children will in turn be indented a further level, and so on.
Lets's try that with a bigger tree, the one I've used as a primary
example.

In [8]:
ntree_2 = tree_to_nodes(
    {1: [2, 3], 2: [4], 4: [5], 5: [6, 7], 3: [8, 9, 10]},
    1
)
print(ntree_2.pretty())

Node 1: ()
| Node 2: ()
| | Node 4: ()
| | | Node 5: ()
| | | | Node 6: ()
| | | | Node 7: ()
| Node 3: ()
| | Node 8: ()
| | Node 9: ()
| | Node 10: ()


It's relatively easy to say that `Node 10` is the third child of `Node 3`, which in turn is the second child of the root.
This nicely-recursive representation makes it relatively easy to write recursive processing agorithms.
Looking at the path down to to `Node 6` makes it fairly obvious there's a string of single-child nodes between
`Nodes 1` and `5`.

We can reshape the tree by stashing inessential nodes in a class attribute. This re-structures the tree,
so technically it woud be better if a) it were a `Tree` method not a `Node` method, and b) it created a completely
new `Tree`. (This prompts a speculation that a `Tree` should really know where it's rooted. But this is early
days yet, and so I'm prepared to put up with these shortcomings, which can be remedied when they block
further progress. We may never get there. This "compression" process is achieved by the `tidied` method

In [9]:
ntree_2 = ntree_2.tidied()
print(ntree_2.pretty())

RecursionError: maximum recursion depth exceeded in comparison